# Decision Tree Learning: College Admission

A decision tree a learned set of rules that allows us to make decisions on data.

We are going to look at the prosper loan dataset.  This dataset shows a history of loans made by Prosper.

In [ ]:
%matplotlib inline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

print('Spark UI running on http://YOURIPADDRESS:' + sc.uiWebUrl.split(':')[2])

## Step 1: Load the Data

In [ ]:
dataset = spark.read.csv("/data/college-admissions/admission-data.csv", 
                         header=True, inferSchema=True)
dataset.show(20)

## Step 2: Build feature vectors using VectorAssembler.

In [ ]:
assembler = VectorAssembler(inputCols=['gpa', 'gre', 'rank'], outputCol="features")
feature_vector = assembler.transform(dataset)
feature_vector = feature_vector.withColumn("label", feature_vector["admit"])
feature_vector.show()

## Step 3: Split Data into training and test.

We will split our the data up into training and test.  (You know the drill by now).

**=> TODO: Split dataset into 80% training, 20% validation**


In [ ]:
# Split the data into training and test sets (20% held out for testing)
(training, test) =  feature_vector.randomSplit([.8,.2])

## Step 4: Decision Tree

In [ ]:
# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxBins=500)
dt

In [ ]:
## train model
model = dt.fit(training)
model

In [ ]:
## Get predictions on test data
predictions = model.transform(test)
predictions.show()

## do a sample label
predictions.sampleBy(col="label", fractions={0: 0.5, 1: 0.5}, seed=123).show()

## Step 5: Evaluate the model.

Let us check to see how the model did, using accuracy as a measure.

In [ ]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("accuracy ",  accuracy)


## Step 6: Do a Few Runs
- Click 'Cell --> Run All'
- Observe how the 'accuracy' output changes above
- Why?  Can we get the same accuracy all the times?